In [16]:
import pickle as pkl
from lda import load_docs_in_bow, process_vectors, compute_word_topic_matrix, compute_document_topic_matrix
from query_ranking import queries_string2int, get_queries, query_score
import numpy as np
from sklearn.metrics import average_precision_score

In [17]:
N_TOPIC = 100
ALPHA = 50 / N_TOPIC
BETA = 0.01

In [18]:
path_to_doc = "../data/docs_trec_covid_selected_docs"

vocabulary, vectors, doc_id = load_docs_in_bow(path_to_doc)

dict_document = process_vectors(vectors)

In [19]:
path_to_learned_data = f'../data/learning_data_covid_{N_TOPIC}_topics.pkl'

with open(path_to_learned_data, 'rb') as file:
    topic_assignment, Mwt, Mdt = pkl.load(file)

In [20]:
for doc, word_topic in topic_assignment.items():
    if len(word_topic) == 0:
        print(doc)

1389
1805
3219


In [21]:
path_qurey = '../data/queries_trec_covid'

int_queries = queries_string2int(get_queries(path_qurey), vocabulary)

queries_scores = []


for i, int_query in enumerate(int_queries):
    print(f'calculate score of {i} query')
    queries_scores.append(query_score(int_query, dict_document, Mwt, Mdt, N_TOPIC, len(vocabulary), ALPHA, BETA))

calculate score of 0 query


KeyboardInterrupt: 

In [ ]:
# import logging
# for n_topic in range(100, 601, 100):
#     logging.debug(f'editing {n_topic} topics file ...')
#     path_to_learned_data = f'../data/learning_data_covid_{n_topic}_topics.pkl'
#     saved_path = f'../data/learning_data_covid_{n_topic}_topics_edited.pkl'
#     
#     with open(path_to_learned_data, 'rb') as file:
#         topic_assignment = pkl.load(file)
#         
#     Mwt = compute_word_topic_matrix(topic_assignment, n_topic, len(vocabulary))
#     Mdt = compute_document_topic_matrix(topic_assignment, n_topic)
#     
#     with open(saved_path, 'wb') as file:
#         pkl.dump((topic_assignment, Mwt, Mdt), file)

In [ ]:
with open('../data/queries_scores_100_topics.pkl', 'wb') as file:
    pkl.dump(queries_scores, file)

In [ ]:
path_to_qrels = "../data/qrels_trec_covid"
with open(path_to_qrels, 'r') as file:
    qrels = file.read().split('\n')[:-1]

dict_qrels = {}
for i in range(0, len(qrels), 3):
    query_id = int(qrels[i])
    doc = qrels[i+1]
    relevance = int(qrels[i+2])
    if doc in doc_id:
        int_doc = doc_id.index(doc)
        if int_doc in dict_qrels:
            dict_qrels[int_doc][query_id] = relevance
        else:
            dict_qrels[int_doc] = {query_id : relevance}

In [ ]:
with open('../data/queries_scores_100_topics.pkl', 'rb') as file:
    queries_scores_100_topics = pkl.load(file)

In [22]:
for q, ranking in enumerate(queries_scores_100_topics):
    top10 = np.argsort(ranking)[::-1][:10]
    y_truth = []
    y_pred = []
    score = 2
    for doc in top10:
        if doc in dict_qrels.keys():
            dict_relevance_doc = dict_qrels[doc]
            if q in dict_relevance_doc.keys():
                y_pred.append(score)
                y_truth.append(dict_relevance_doc[q])
        
        score -= 0.2
        

KeyboardInterrupt: 

In [36]:
print(dict_qrels[5])

{1: 0, 2: 0, 12: 0, 15: 0, 29: 0}


In [49]:
def average_precision(query, scores, dict_qrels):
    ranking = np.argsort(scores)[::-1]
    
    n_relevant = 0
    sum_precision = 0.0
    
    for rank, doc in enumerate(ranking, start=1):
        if dict_qrels[doc].get(query, 0) > 0:
            n_relevant += 1
            sum_precision += n_relevant / rank
    
    if n_relevant == 0:
        return 0

    return sum_precision / n_relevant

In [50]:
average_precision_list = [average_precision(query, scores, dict_qrels) for query, scores in enumerate(queries_scores_100_topics)]

print(sum(average_precision_list) / len(average_precision_list))

0.016792510172324318
